In [7]:
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from sklearn.metrics import roc_auc_score,confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
import datetime
from pandas import Series
from collections import Counter
from sklearn.externals import joblib

In [2]:
file_1 = pd.read_csv('open_data_train_valid/train/train_1.txt',chunksize=100,sep='\t')
file_1 = file_1.get_chunk()
columns = file_1.columns
#train data
print('load training data')


table_1 = pd.read_table('open_data_train_valid/train/train_1.txt', sep='\t', chunksize=20000)
table_2 = pd.read_table('open_data_train_valid/train/train_2.txt', sep='\t', chunksize=20000, names=columns)
table_1 = table_1.get_chunk()
table_2 = table_2.get_chunk()
data = pd.concat([table_1, table_2], axis=0, ignore_index=True)
label = data['label']
label = label.iloc[label[label.notnull()].index.values]
data = data.iloc[label[label.notnull()].index.values]
data.drop(['label'], axis=1, inplace=True)
data.drop(['loan_dt','tag','id'],axis=1,inplace = True)

load training data


In [3]:
print('load valid set')
valid_set = pd.read_csv('open_data_train_valid/valid.txt',sep='\t',chunksize = 20000)
valid_set = valid_set.get_chunk()
valid_set.drop(['loan_dt','id'],axis=1,inplace = True)

load valid set


In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV,cross_val_predict   #Perforing grid search

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [5]:
def xgb_feature(X_train, y_train, X_test, y_test=None,valid = True):
    # 模型参数
    model = XGBClassifier(
 learning_rate =0.05,
 n_estimators=550,
 max_depth=3,
 min_child_weight=0.8,
 gamma=0.005,
 reg_alpha = 1e-5,
 reg_lambda = 4,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
    silent = False)
    if valid:
        print('xgboost 1')
        model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_train, y_train)],
                  eval_metric='auc', verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
    else:
        model.fit(X_train, y_train, verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
        #print('AUC: %.4f' % roc_auc_score(y_test, predict))
    return predict

def xgb_feature2(X_train, y_train, X_test, y_test=None,valid = True):
    # 模型参数
    model = XGBClassifier(
 learning_rate =0.05,
 n_estimators=600,
 max_depth=3,
 min_child_weight=0.8,
 gamma=0.005,
 reg_alpha = 1e-5,
 reg_lambda = 2,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=4,
    silent = False)
    if valid:
        print('xgboost 2')
        model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_train, y_train)],
                  eval_metric='auc', verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
    else:
        model.fit(X_train, y_train, verbose=True)
        #model.fit(X_train, y_train, early_stopping_rounds=15, eval_set=[(X_train, y_train), (X_test, y_test)],
        #          eval_metric='auc', verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
        #print('AUC: %.4f' % roc_auc_score(y_test, predict))
    return predict

def xgb_feature3(X_train, y_train, X_test, y_test=None,valid=True):
    # 模型参数
    model = XGBClassifier(
 learning_rate =0.05,
 n_estimators=550,
 max_depth=3,
 min_child_weight=0.8,
 gamma=0.005,
 reg_alpha = 1,
 reg_lambda = 5,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=5,
    silent = False)
    if valid:
        print('xgboost 3')
        model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_train, y_train)],
                  eval_metric='auc', verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
    else:
        model.fit(X_train, y_train, verbose=True)
        #model.fit(X_train, y_train, early_stopping_rounds=15, eval_set=[(X_train, y_train), (X_test, y_test)],
        #          eval_metric='auc', verbose=True)
        predict = model.predict_proba(X_test)[:, 1]
        #print('AUC: %.4f' % roc_auc_score(y_test, predict))
    return predict

In [8]:
# heamy
dataset = Dataset(X_train=data, y_train=label, X_test=valid_set, y_test=None, use_cache=False)
model_xgb = Classifier(dataset=dataset, estimator=xgb_feature, name='xgb', use_cache=False)
model_xgb2 = Classifier(dataset=dataset, estimator=xgb_feature2, name='xgb2', use_cache=False)
model_xgb3 = Classifier(dataset=dataset, estimator=xgb_feature3, name='xgb3', use_cache=False)

In [9]:
pipeline = ModelsPipeline(model_xgb, model_xgb2, model_xgb3)
print('stacking model')
stack_ds = pipeline.stack(k=2, add_diff=False, full_test=True)

stacking model
xgboost 1
[23:49:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	validation_0-auc:0.694445
Will train until validation_0-auc hasn't improved in 10 rounds.
[23:49:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	validation_0-auc:0.727983
[23:49:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	validation_0-auc:0.739608
[23:49:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[3]	validation_0-auc:0.759663
[23:49:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[4]	validation_0-auc:0.781127
[23:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[5]	validation_0-auc:0.78308
[23:49:31] src/tree/updater_prune.cc:74: tree pruning end, 1

[56]	validation_0-auc:0.844057
[23:52:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[57]	validation_0-auc:0.845204
[23:52:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[58]	validation_0-auc:0.846031
[23:52:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[59]	validation_0-auc:0.846452
[23:52:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[60]	validation_0-auc:0.848367
[23:52:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[61]	validation_0-auc:0.849294
[23:52:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[62]	validation_0-auc:0.84993
[23:52:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3


[23:54:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[115]	validation_0-auc:0.890537
[23:54:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[116]	validation_0-auc:0.891361
[23:54:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[117]	validation_0-auc:0.891998
[23:54:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[118]	validation_0-auc:0.892653
[23:54:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[119]	validation_0-auc:0.89302
[23:54:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[120]	validation_0-auc:0.893568
[23:54:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[121]	validation_0-auc:0.

[23:56:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[172]	validation_0-auc:0.919112
[23:56:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[173]	validation_0-auc:0.919519
[23:56:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[174]	validation_0-auc:0.920124
[23:56:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[175]	validation_0-auc:0.920601
[23:56:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[176]	validation_0-auc:0.920918
[23:56:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[177]	validation_0-auc:0.921437
[23:56:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[178]	validation_0-auc:0

[23:58:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[229]	validation_0-auc:0.940227
[23:58:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[230]	validation_0-auc:0.94048
[23:58:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[231]	validation_0-auc:0.94077
[23:58:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[232]	validation_0-auc:0.941198
[23:58:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[233]	validation_0-auc:0.941491
[23:58:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[234]	validation_0-auc:0.94183
[23:58:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[235]	validation_0-auc:0.94

...

[08:16:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[306]	validation_0-auc:0.930762
[08:16:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[307]	validation_0-auc:0.930991
[08:17:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[308]	validation_0-auc:0.93122
[08:17:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[309]	validation_0-auc:0.931364
[08:17:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[310]	validation_0-auc:0.931625
[08:17:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[311]	validation_0-auc:0.931813
[08:18:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[312]	validation_0-auc:0.

[08:32:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[363]	validation_0-auc:0.940967
[08:32:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[364]	validation_0-auc:0.941099
[08:32:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[365]	validation_0-auc:0.941163
[08:33:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[366]	validation_0-auc:0.941321
[08:33:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[367]	validation_0-auc:0.94143
[08:33:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[368]	validation_0-auc:0.941642
[08:33:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[369]	validation_0-auc:0.

[08:45:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[420]	validation_0-auc:0.949988
[08:46:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[421]	validation_0-auc:0.95015
[08:46:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[422]	validation_0-auc:0.950335
[08:46:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[423]	validation_0-auc:0.950406
[08:46:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[424]	validation_0-auc:0.950546
[08:46:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[425]	validation_0-auc:0.950685
[08:47:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[426]	validation_0-auc:0.

[08:57:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[477]	validation_0-auc:0.95707
[08:58:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[478]	validation_0-auc:0.957208
[08:58:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[479]	validation_0-auc:0.957381
[08:58:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[480]	validation_0-auc:0.957541
[08:58:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[481]	validation_0-auc:0.957686
[08:58:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[482]	validation_0-auc:0.957801
[08:59:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[483]	validation_0-auc:0.

[09:09:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[534]	validation_0-auc:0.963649
[09:09:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[535]	validation_0-auc:0.963705
[09:10:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[536]	validation_0-auc:0.963746
[09:10:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[537]	validation_0-auc:0.963814
[09:10:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[538]	validation_0-auc:0.963965
[09:10:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[539]	validation_0-auc:0.96407
[09:10:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[540]	validation_0-auc:0.

In [35]:
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression,
                     parameters={'C': 1})
predict_result = stacker.predict()

In [36]:
np.save(predict_result.

array([0.0407505 , 0.03207151, 0.02586532, ..., 0.11579423, 0.23716817,
       0.04285989])

In [19]:
def write_file(filename, proba):
    with open(filename, 'w') as f:
        f.write('id,prob')
        f.write('\n')
        for i in range(proba.shape[0]):
            line = str(100000 + i + 1) + ',' + str(proba[i])
            f.write(line + '\n')

In [29]:
write_file('valid_data/{}.txt'.format(datetime.datetime.now().strftime('%F %T'))
            , predict_result)

In [ ]:
predict_result = np.array(stack_ds.X_test.mean(axis=1))

In [ ]:
stacker.stack()

In [10]:
stack_ds.X_test.to_csv()

,xgb,xgb2,xgb3
0,0.027263,0.119146,0.113388
1,0.013578,0.053959,0.085248
2,0.008378,0.033242,0.034703
3,0.016623,0.030890,0.039287
4,0.229453,0.469205,0.451607
5,0.008621,0.020039,0.025511
6,0.030638,0.114511,0.131878
7,0.372342,0.669343,0.702329
8,0.025365,0.077983,0.103774
9,0.045689,0.151590,0.218501


In [14]:
predict_result = np.array(stack_ds.X_test.mean(axis=1))

In [21]:
dis = np.array([0.5,0.3,0.2])

In [23]:
predict_result = np.array(stack_ds.X_test).dot(dis)

In [27]:
predict_result.sum()

2748.9765477284036

In [28]:
predict_result

array([0.07205306, 0.04002626, 0.0211021 , ..., 0.19082998, 0.31619455,
       0.08106285])